#### ADBMS LAB 9
##### NAME : Ashutosh Singh Kushwaha
##### Admission N0 :22MT0084

#### Objective : 
Create a movie ticket booking system on python which is connected to a database. The users should
be able to choose the movie, theater, screen and city of their choice to watch the movie. Use python
for your work and use object oriented features of python like classes, inheritance, etc. Then input
these values into a connected database. Additionally use git for version control.

In [1]:
import mysql.connector as sc

In [2]:
myconn = sc.connect( 
    database = "adbms",
    host = "localhost",
    password = "Kushashu123",
    user = "root"
)
print(myconn)

In [3]:
cursor = myconn.cursor(buffered = True)

In [4]:
query = """show tables"""
cursor.execute(query)
for x in cursor:
    print(x)

('booking',)
('city',)
('department_salary_total',)
('dept_avg',)
('dp',)
('faculty',)
('instructor',)
('movie',)
('shows',)
('teaches',)
('teaches2',)
('theater',)
('user_detail',)
('user_view',)


##### We are Creating five table 
1. Movie 
2. Theater
3. City
4. Current running shows
5. Booking 

In [53]:
## MOVIE
query = """create table movie (
movie_id Integer primary key,
Name varchar(40) not null,
Genre varchar(40) not null,
duration Integer not null,
check ((duration between 15 and 200)) )"""
cursor.execute(query)

In [54]:
## Theater
query = """create table theater(
theater_id Integer primary key,
Name varchar(40) not null,
capacity Integer not null,
filled Integer not null,
pincode Integer,
screen Integer,
foreign key(pincode) references city(pincode),
check (filled>-1 and capacity>0))"""
cursor.execute(query)

In [55]:
## City 
query = """create table city(
pincode integer primary key,
Name varchar(30) not null)"""
cursor.execute(query)

In [56]:
## Current Running Shows
query = """create table shows(
movie_id integer,
theater_id integer,
screen integer,
foreign key(movie_id) references movie(movie_id),
foreign key(theater_id) references theater(theater_id)
)"""
cursor.execute(query)

In [57]:
## booking
query = """create table booking(
user_id integer primary key,
user_name varchar(30),
movie_name varchar(30),
theater_name varchar(30),
screen integer,
city varchar(30),
seat_no integer
)"""
cursor.execute(query)

In [58]:
### inserting  Movie
query = """insert into movie values(%s,%s,%s,%s)"""
cursor.execute(query,(1,"Bhool Bhulaiya","comedy",154))
cursor.execute(query,(2,"Bhool Bhulaiya 2","comedy",130))
cursor.execute(query,(3,"pathaan","action",146))
cursor.execute(query,(4,"Drishayam","thriller",130))

In [59]:
## Adding theaters
query = """insert into theater values(%s,%s,%s,%s,%s,%s)"""
cursor.execute(query,(1, 'Carnival', 120, 0, 208007, 1))
cursor.execute(query,(2, 'Carnival', 120, 0, 208007, 2))
cursor.execute(query,(3, 'PVR', 120, 0, 208007, 1))
cursor.execute(query,(4, 'PVR', 120, 0, 208007, 2))
cursor.execute(query,(5, 'PVR', 120, 0, 208007, 3))
cursor.execute(query,(6,"INOX",120,0,826004,1))
cursor.execute(query,(7,"INOX",120,0,826004,2))

In [60]:
## Creating show 
query = """insert into shows values(%s,%s,%s)"""
cursor.execute(query ,(1,1,1))
cursor.execute(query ,(2,2,2))
cursor.execute(query ,(1,3,1))
cursor.execute(query ,(1,4,2))
cursor.execute(query ,(2,5,3))

In [ ]:
### Defining Movie Class with 

In [25]:
class movie_portal:
    def book(self):
         ## Booking 
        query = "select * from booking"
        cursor.execute(query)
        user_id = cursor.rowcount+1
        city = input("Enter Your City   ").lower()
        theater = input("Enter Theater Name   ").lower()
        movie = input("Enter movie Name   ").lower()
        screen = int(input("Enter Screen No   "))
        isavailable,details = self.show_available(movie,theater,screen,city)
        if isavailable:
            user_name = input("Enter Your Name   ")
            if(self.book_ticket(details,user_id,user_name)):
                print("BooKing Successfull")
            else:
                print("ERROR") 
        else:
            print("__________________Sorry! No Show Available__________________")
    def __init__(self):
        print("\n______________________________________________________Welcome in Movie portal________________________________________________________")
        ans = input("\n Select your Choice \n\n1.Book Ticket \n2.Booking History \n3.Exit\n")
        if ans =='1':
            print("\n\n__________________________________________________________Available shows_________________________________________________________")
            self.Running_show()
            self.book()
        elif ans =='2':
            self.booking_history()
        else:
            print("Good Bye")
        
        print("\n\n\n________________________________________________________________Thanks________________________________________________________________")
        
        
    def Running_show(self):
        print("||","Movie".center(20," "),"||","Duration".center(8," "),"||","Theater Name".center(20," "),"||","City".center(20," "),"||","Total Seats".center(12," "),"||","Available".center(12," "),"||","Screen No".center(12," "),"||")
        query = """select m.name,m.duration,t.name,c.name,t.capacity,(t.capacity-t.filled),t.screen
        from movie m,theater t ,shows s ,city c where s.movie_id = m.movie_id 
        and t.theater_id = s.theater_id and t.pincode = c.pincode and s.screen=t.screen and
        (t.capacity-t.filled)>0"""
        cursor.execute(query)
        for x in cursor:
            print("||",x[0].center(20," "),"||",str(x[1]).center(8," "),"||",x[2].center(20," "),"||",x[3].center(20," "),"||",str(x[4]).center(12," "),"||",str(x[5]).center(12," "),"||",str(x[6]).center(12," "),"||")    
    ### isshow available
    
    def show_available(self,movie,theater,screen,city):
        query = """select m.name,m.movie_id,t.name,t.theater_id,c.name,c.pincode,t.filled,
        (t.capacity-t.filled),t.screen from movie m,theater t ,shows s ,city c where s.movie_id = m.movie_id 
        and t.theater_id = s.theater_id and t.pincode = c.pincode and s.screen=t.screen and
        (t.capacity-t.filled)>0"""
        cursor.execute(query)
        for x in cursor:
            if(x[0].lower()==movie and x[2].lower()==theater and x[7]>0 and x[8]==screen and x[4].lower()==city):
                return (True ,x)
        return (False,(0))
    
    ### Updating available seats
    def update_table(self,id,remaining):
        query = """update theater set filled = (%s) where theater_id = (%s)"""
        cursor.execute(query ,(remaining,id))
        
    ### Perform booking 
    def book_ticket(self,details,user_id ,user_name):
        self.update_table(details[3],details[6]+1)
        query = "insert into booking values(%s,%s,%s,%s,%s,%s,%s)"
        cursor.execute(query,(user_id,user_name,details[0],details[2],details[8],details[4],details[6]+1))
        return True
    def booking_history(self):
        query = """select * from booking"""
        cursor.execute(query)
        print("\n\n\n___________________________________________________________Booking History___________________________________________________________")
        print("\n\n||","User ID".center(10," "),"||","User Name".center(20," "),"||","Movie Name".center(20," "),"||","Theater Name".center(12," "),"||","Screen No".center(12," "),"||","City".center(12," "),"||","Seat No".center(8," "),"||")
        for x in cursor:
            print("||",str(x[0]).center(10," "),"||",x[1].center(20," "),"||",x[2].center(20," "),"||",x[3].center(12," "),"||",str(x[4]).center(12," "),"||",x[5].center(12," "),"||",str(x[6]).center(8," "),"||") 
            

In [29]:
## main Driver code 
m = movie_portal()


______________________________________________________Welcome in Movie portal________________________________________________________

 Select your Choice 

1.Book Ticket 
2.Booking History 
3.Exit
2



___________________________________________________________Booking History___________________________________________________________


||  User ID   ||      User Name       ||      Movie Name      || Theater Name ||  Screen No   ||     City     || Seat No  ||
||     1      ||         Ashu         ||    Bhool Bhulaiya    ||   Carnival   ||      1       ||    Kanpur    ||    2     ||
||     2      ||        suman         ||   Bhool Bhulaiya 2   ||     PVR      ||      3       ||    Kanpur    ||    1     ||
||     3      ||      Alok Kumar      ||    Bhool Bhulaiya    ||     PVR      ||      1       ||    Kanpur    ||    1     ||



________________________________________________________________Thanks________________________________________________________________


In [30]:
m = movie_portal()


______________________________________________________Welcome in Movie portal________________________________________________________

 Select your Choice 

1.Book Ticket 
2.Booking History 
3.Exit
1


__________________________________________________________Available shows_________________________________________________________
||        Movie         || Duration ||     Theater Name     ||         City         || Total Seats  ||  Available   ||  Screen No   ||
||    Bhool Bhulaiya    ||   154    ||       Carnival       ||        Kanpur        ||     120      ||     118      ||      1       ||
||    Bhool Bhulaiya    ||   154    ||       Carnival       ||        Kanpur        ||     120      ||     118      ||      1       ||
||    Bhool Bhulaiya    ||   154    ||       Carnival       ||        Kanpur        ||     120      ||     120      ||      2       ||
||   Bhool Bhulaiya 2   ||   130    ||       Carnival       ||        Kanpur        ||     120      ||     120      ||      2 